In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import markdown
import re


C:\Users\bOct0\AppData\Local\Temp\ipykernel_13328\2434883213.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
excel_file = 'kfk.xlsx'
df = pd.read_excel(excel_file)

In [3]:
input_string = df['response'][1299]
print(input_string)

> **Test Case 1**
> * **Description:** Contact a repository that does not exist
> * **Input:**
>     * Repository name: `non-existent-repository`
> * **Expected Output:**
>     * Error message indicating that the repository does not exist
> 
> **Test Case 2**
> * **Description:** Contact a repository with a valid name
> * **Input:**
>     * Repository name: `my-repository`
> * **Expected Output:**
>     * Response containing information about the repository, such as its description, contributors, and recent commits
> 
> **Test Case 3**
> * **Description:** Contact a private repository without proper authentication
> * **Input:**
>     * Repository name: `private-repository`
> * **Expected Output:**
>     * Error message indicating that authentication is required
> 
> **Test Case 4**
> * **Description:** Contact a public repository with proper authentication
> * **Input:**
>     * Repository name: `public-repository`
> * **Expected Output:**
>     * Response containing information about

In [4]:
html = markdown.markdown(input_string)
text = BeautifulSoup(html, features='html.parser')
text = text.get_text().strip().replace('\n','')
text = text.strip().replace('*','')
res = re.sub(' +', ' ', text)
res

'Test Case 1 Description: Contact a repository that does not exist Input: Repository name: non-existent-repository Expected Output: Error message indicating that the repository does not existTest Case 2 Description: Contact a repository with a valid name Input: Repository name: my-repository Expected Output: Response containing information about the repository, such as its description, contributors, and recent commitsTest Case 3 Description: Contact a private repository without proper authentication Input: Repository name: private-repository Expected Output: Error message indicating that authentication is requiredTest Case 4 Description: Contact a public repository with proper authentication Input: Repository name: public-repository Expected Output: Response containing information about the repository, even though it is publicTest Case 5 Description: Contact a repository and specify a specific commit SHA Input: Repository name: my-repository Commit SHA: 1234567890abcdef Expected Output

In [5]:

# Split the string into individual test cases
test_cases = res.split("Test Case")[1:]

# Initialize lists to store data
descriptions = []
inputs = []
expected_outputs = []

# Extract information for each test case
for test_case in test_cases:
    parts = test_case.split("Description:")
    description = parts[1].split("Input:")[0].strip()
    input_part = parts[1].split("Input:")[1]
    input_data = input_part.split("Expected Output:")[0].strip()
    expected_output = input_part.split("Expected Output:")[1].strip()
    
    descriptions.append(description)
    inputs.append(input_data)
    expected_outputs.append(expected_output)

# Create a dataframe
data = {
    "User Story":df['userstories'][1000],
    "Description": descriptions,
    "Input": inputs,
    "Expected Output": expected_outputs
}

dt = pd.DataFrame(data)

dt

,User Story,Description,Input,Expected Output
0,I want to receive suggestions about local spo...,Contact a repository that does not exist,Repository name: non-existent-repository,Error message indicating that the repository d...
1,I want to receive suggestions about local spo...,Contact a repository with a valid name,Repository name: my-repository,Response containing information about the repo...
2,I want to receive suggestions about local spo...,Contact a private repository without proper au...,Repository name: private-repository,Error message indicating that authentication i...
3,I want to receive suggestions about local spo...,Contact a public repository with proper authen...,Repository name: public-repository,Response containing information about the repo...
4,I want to receive suggestions about local spo...,Contact a repository and specify a specific co...,Repository name: my-repository Commit SHA: 123...,Response containing information about the spec...
